# Create Conveyor device (automate_process)

This module create a Conveyor device, its correponding antennas (as povs) and link them to a fixture.

Module interaction:

- Environement selection
- Premise selection
- Floor selection
- Area selection
- Zone selection
- Device model selection
- => device creation
- - - ID : identifier of the device (unsigned integer), must be unique in the organization. Auto generated by default
- - - IP (mandatory) : ip address of the device on the local network, must be unique in the premise (unique in the organization for ShopCX)
- - - NAME (mandatory) : friendly name of the device
- - - MAC : MAC address of the device
- POV creation:
- - Select antenna model 
- - Update parameters:
- - - ID : identifier of the pov (unsigned integer), must be unique in the organization. Auto generated by default
- - - Addr slots (mandatory) : Antenna slot of the pov on the device
- - - NAME (mandatory) : friendly name of the pov
- - => pov creation and link to device 
- - Fixture selection
- - => fixture link to pov 


## Select an environment

In [ ]:
%load_ext autoreload
%autoreload 2
from libConfUtils import chooseEnv as chooseEnv
from libConfUtils import deviceUtils as deviceUtils
from libConfUtils import povUtils as povUtils
from libConfUtils import uiUtils as uiUtils
from libConfUtils import premiseNhubUtils as premiseNhubUtils
from libConfUtils import settingsUtils as settingsUtils

import json
import ipaddress

In [ ]:
%%capture
status = True
myEnv = chooseEnv.choosePremise()

In [ ]:
if status:
    print("Organization: " + myEnv.get('urlDP'))

## Create device

In [ ]:
if status:
    myEnv = chooseEnv.chooseDeviceModel(myEnv, 'AUTOMATED_PROCESS');
    deviceModel = myEnv['deviceModel'];
    if not deviceModel:
        text = 'No Conveyor Model Found '
        uiUtils.displayMessage('Conveyor creation',text)
        print(text)
        status = False
    else:
        print("Device Model ID: " + deviceModel['id'])    

In [ ]:
if status:
    print("Device Model ID: " + deviceModel['id'])

In [ ]:
%%capture
if status:
    myForm = uiUtils.inputWithForm('Conveyor definition',['IP*','NAME*','MAC'])

    did = '' # myForm.get('ID');
    ip = myForm.get('IP*');
    name = myForm.get('NAME*');
    mac = myForm.get('MAC');
           
    if not name:
        text = 'Conveyor Name is a mandatory fied '
        uiUtils.displayMessage('Conveyor creation',text)
        print(text)
        status = False
        
    try:    
        tmp = ipaddress.ip_address(ip)
    except:
        text = 'Conveyor IP address is not correct '
        uiUtils.displayMessage('Conveyor creation',text)
        print(text)
        status = False        
    

In [ ]:
#%% CHECK ID UNICITY : deprecated
checkIpUnicity = True

In [ ]:
#%% CHECK IP UNICITY in the Premise
# DEV NOTE : should be integrated in the backEnd
if status and checkIpUnicity:
    res = deviceUtils.checkIPUnicityOnThePremise(myEnv,ip)
    if not res.get('status'):
        text = res.get('message')
        text  = text + '\n'+ ' Conveyor can not been created'
        uiUtils.displayMessage('Conveyor create',text)
        status = False

In [ ]:
if status :
    deviceId = deviceUtils.createCONVEYOR(myEnv, ip, name, did, mac, deviceModel=deviceModel['id']);
    if not deviceId:
        text = 'Conveyor Creation: Error during creation of the device '
        uiUtils.displayMessage('Conveyor creation',text)
        print(text)
        status = False

In [ ]:
if status:
    print("Conveyor ID: " + deviceId)

## Create POVs

In [ ]:
%%capture
if status:
    status = False
    retryNb = 3
    while (retryNb>0):
        myForm = uiUtils.inputWithForm('How many pov to add to this reader?',['nb*(integer)'])
        nbPovStr = myForm.get('nb*(integer)')
        if nbPovStr and not nbPovStr.isdigit():
            text = 'Conveyor Creation: Number of PointOfView must an interger, not ' + str(nbPovStr)
            uiUtils.displayMessage('Conveyor creation',text)
            print(text)
            retryNb = retryNb-1
        else:
            nbOfPov = int(nbPovStr);
            status = True
            break;

In [ ]:
%%capture
if status:
    device = deviceUtils.getDeviceById(myEnv, deviceId);
    if not device:
        text = 'Conveyor Creation: Can not retrieve automated process device ' + deviceId
        uiUtils.displayMessage('Conveyor creation',text)
        print(text)
        status = False

if status:        
    deviceTypeId = device['deviceType']['id']
    if not deviceTypeId:
        text = 'Conveyor Creation: Can not retrieve automated process device type of ' + str(device)
        uiUtils.displayMessage('Conveyor creation',text)
        print(text)
        status = False

In [ ]:
%%capture
if status :
    povs = []
    for x in range(nbOfPov):
        retryCount = 3
        
        while (retryCount >0):
            myEnv = chooseEnv.choosePovModel(myEnv, deviceTypeId)
            povModel = myEnv['povModel']

            myForm = uiUtils.inputWithForm('Conveyor POV definition',['AntennaSlots*','NAME*'])
            did = '' #myForm.get('ID')
            slots = myForm.get('AntennaSlots*')
            name = myForm.get('NAME*')

            if not povModel:
                text = 'Conveyor PoV Model is a mandatory fied '
                uiUtils.displayMessage('Conveyor creation',text)
                print(text)
                retryCount = retryCount-1
            elif not slots:
                text = 'Conveyor PoV slots is a mandatory fied '
                uiUtils.displayMessage('Conveyor creation',text)
                print(text)
                retryCount = retryCount-1

            elif not name:
                text = 'Conveyor PoV Name is a mandatory fied '
                uiUtils.displayMessage('Conveyor creation',text)
                print(text)
                retryCount = retryCount-1

            else:
                status = True
                break;
        # end of while
        if status:
            povs.append((slots, name, did, povModel['id']))

In [ ]:
if status:
    text = json.dumps(povs, indent=4, sort_keys=True)
    uiUtils.displayMessage('PassThough creation',text)
    print(text)

In [ ]:
if status:
    ids = []
    for pov in povs:
        ids.append(povUtils.createPov(myEnv, *pov))
    if not ids:
        text = 'Conveyor Creation: Error during PoV creation '
        uiUtils.displayMessage('Conveyor creation',text)
        print(text)
        status = False

In [ ]:
if status:
    for povId in ids:
        try:
            povUtils.linkPovToDevice(myEnv, povId, deviceId)
        except:
            text = 'Conveyor Creation: Error during PoV '+str(povId)+' link to device '
            uiUtils.displayMessage('Conveyor creation',text)
            print(text)

## Link to fixtures

In [ ]:
if status:
    myEnv = chooseEnv.chooseZone(myEnv);
    myEnv = chooseEnv.chooseFixture(myEnv, dialogTitle='Select fixture for all pov ')
    for povId in ids:
        try:
            povUtils.linkPovToFixture(myEnv, povId, myEnv['FixtureId'])
        except:
            text = 'Conveyor Creation: Error during PoV '+str(povId)+' link to fixture '
            uiUtils.displayMessage('Conveyor creation',text)
            print(text)

## Validate device has been created with default settings and start mode

### Get XPoint (with all link resolved)

In [ ]:
if status:
    result = deviceUtils.getDeviceById(myEnv, deviceId);

In [ ]:
if status:
    text = json.dumps(result, indent=4, sort_keys=True)
    uiUtils.displayMessage('Conveyor creation',text)
    print(text)

### Get RFID settings

In [ ]:
if status:
    settingsUtils.getRfidSettingsByDeviceId(myEnv, result['device']['id'], 'AUTOMATED_PROCESS')

### Get linked POVs

In [ ]:
%%capture
if status:
    settings = {}
    for pov in result['devicePov']:
        povId = pov['id']
        settings[povId + '_' + 'AUTOMATED_PROCESS'] = settingsUtils.getRfidSettingsByPovId(myEnv, povId,'AUTOMATED_PROCESS')

In [ ]:
if status:
    text = json.dumps(settings, indent=4, sort_keys=True)
    uiUtils.displayMessage('Conveyor creation',text)
    print(text)